# Old notebook

This notebook has content that is no longer part of the main branch of development of the project, although it still has code sections that may be useful in the future. For this reason, it is saved in a sub-folder of archived noetbooks.